# Action Data
### Ignore this unless you want to edit Action information.

In [66]:
from dataclasses import dataclass 

# Define the shape of an Action
@dataclass
class Action:
    potency: int = 0
    heat: int = 0
    battery: int = 0
    onGCD: bool = False
    recast: int = 2500
    lock: int = 750
    cooldown: int = 0

In [67]:
# Data entry
DRILL = Action(potency = 700, onGCD = True, cooldown = 20000)
AA = Action(potency = 700, battery = 20, onGCD = True, cooldown = 40000)
CS = Action(potency = 1000, battery = 20, onGCD = True, cooldown = 60000)
SPLIT = Action(potency = 220, heat = 5, onGCD = True)
SLUG = Action(potency = 330, heat = 5, onGCD = True)
CLEAN = Action(potency = 440, heat = 5, battery = 10, onGCD = True)
HB = Action(potency = 220, onGCD = True, recast = 1500)
GR = Action(potency = 150, cooldown = 15000)
RICO = Action(potency = 150, cooldown = 15000)
WF = Action(potency = 1200, cooldown = 120000)
BS = Action(heat = 50, cooldown = 120000)
REA = Action(cooldown = 55000)
HC = Action()

# Buff Data
### Ignore this unless you want to edit buff timings or multipliers/crit/DH rates.

In [50]:
# Define the shape of a Buff
@dataclass
class Buff:
    start: int
    duration: int
    multiplier: float = 1
    critRate: float = 0
    dhRate: float = 0

In [51]:
# Raid buff data entry
TA = Buff(start = 8600, duration = 15000, multiplier = 1.05)
DIV = Buff(start = 10700, duration = 15000, multiplier = 1.06)
EMBOLDEN = Buff(start = 8300, duration = 20000, multiplier = 1.2)  # Decays
RF_1 = Buff(start = 8000, duration = 15000, multiplier = 1.02)
RF_3 = Buff(start = 8000, duration = 15000, multiplier = 1.05)
BH = Buff(start = 9100, duration = 15000, multiplier = 1.05)
DS = Buff(start = 3600, duration = 20000, multiplier = 1.05)
TF = Buff(start = 7500, duration = 20000, multiplier = 1.05)
BL = Buff(start = 2400, duration = 20000, critRate = 0.1)
CHAIN = Buff(start = 8200, duration = 15000, critRate = 0.1)
BV = Buff(start = 3900, duration = 20000, dhRate = 0.2)
DM = Buff(start = 10000, duration = 20000, critRate = 0.2, dhRate = 0.2)

RAID_BUFFS = {
    'Trick Attack': TA,
    'Divination': DIV,
    'Embolden': EMBOLDEN,
    'Radiant Finale (1 Coda)': RF_1,
    'Radiant Finale (3 Codas)': RF_3,
    'Brotherhood': BH,
    'Dragon Sight': DS,
    'Technical Finish': TF,
    'Battle Litany': BL,
    'Chain Stratagem': CHAIN,
    'Battle Voice': BV,
    'Devilment': DM,
}

# Opener Data
### Ignore this unless you want to create a new opener for comparison.

In [52]:
# Data entry
FOURTH_GCD = [REA, DRILL, GR, RICO, SPLIT, BS, SLUG, GR, RICO, AA, WF, HC, HB, GR, HB, RICO, HB, GR, HB, RICO, CLEAN, GR, RICO, DRILL, RICO, SPLIT, SLUG, CLEAN]

# Add rotations here
OPENERS_TO_COMPARE = [
    FOURTH_GCD,
]

# Logic
### Ignore this unless you want to change how the rotations are evaluated.

In [61]:
from dataclasses import replace
from typing import List

DH_MULTIPLIER = 1.25

@dataclass
class Stats:
    critMod: float
    critRate: float
    dhRate: float
    multiplier: float = 1

    def __iadd__(self, buff: Buff):
        """Overloads += to combine base stats + buffs easily"""
        self.critRate += buff.critRate
        self.dhRate += buff.dhRate
        self.multiplier *= buff.multiplier
        return self

class Simulate:
    rotation: List[Action]
    buffs: List[Buff]
    stats: Stats

    # State
    hypercharges: int = 0
    reassembled: bool = False
    time: int = 0
    nextGCD: int = 0
    nextAction: int = 0
    totalEffectivePotency: int = 0
    heat: int = 0
    battery: int = 0

    def __init__(self, rotation: List[Action], buffs: List[Buff], stats: Stats):
        self.rotation = rotation
        self.buffs = buffs
        self.stats = stats
        self.simulate()

    def onReassemble(self):
        self.reassembled = True

    def onHypercharge(self):
        self.hypercharges = 5

    def collectBuffs(self) -> Stats:
        currentStats = replace(self.stats)

        for buff in self.buffs:
            if self.time >= buff.start and self.time < buff.start + buff.duration:
                currentStats += buff

        return currentStats

    def onDamage(self, action: Action):
        potency = action.potency
        stats = self.collectBuffs()
        critMod = stats.critMod
        critRate = stats.critRate
        dhRate = stats.dhRate

        if action.onGCD:
            if self.reassembled:
                critRate = 1
                dhRate = 1
                self.reassembled = False
            
            if self.hypercharges:
                potency += 20
                self.hypercharges -= 1

            self.nextGCD = self.time + action.recast

        elif action == WF:
            critRate = 0
            dhRate = 0

        # Make sure these cap at 100%
        critRate = min(critRate, 1)
        dhRate = min(dhRate, 1)

        effectiveMultiplier = (1 + ((critMod - 1) * critRate)) * (1 + ((DH_MULTIPLIER - 1) * dhRate)) * stats.multiplier
        self.totalEffectivePotency += effectiveMultiplier * potency

    def simulate(self):
        for position, action in enumerate(self.rotation):
            if action.potency:
                self.onDamage(action)
            
            elif action == REA:
                self.onReassemble()

            elif action == HC:
                self.onHypercharge()

            self.heat += action.heat
            self.battery += action.battery

            if action.onGCD:
                self.time = max(self.time + action.lock, self.nextGCD)

            # ALlow for prepull reassemble
            elif not (action == REA and position == 0):
                self.time += action.lock

    def getEffectivePotency(self) -> int:
        return self.totalEffectivePotency

    def getEffectivePPS(self) -> float:
        return (self.totalEffectivePotency / self.time) * 1000

# Analysis
### Choose which buffs to compare and see the results.

In [54]:
import ipywidgets as widgets
from IPython.display import display

buffs = RAID_BUFFS.keys()
checkboxes = [ widgets.Checkbox(value=False, description=buff) for buff in buffs ]
checkboxObject = widgets.VBox(children=checkboxes)

print("Raid Buffs")
display(checkboxObject)

Raid Buffs


In [64]:
selectedBuffs = []

# Collect the buffs toggled above
for c in checkboxes:
    if c.value:
        selectedBuffs.append(RAID_BUFFS[c.description])

for opener in OPENERS_TO_COMPARE:
    data = Simulate(opener, selectedBuffs, Stats(1.6, 0.25, 0.33, 1))
    print(data.getEffectivePPS())

306.7472019551283
